<a href="https://colab.research.google.com/github/tmoura/softexIA/blob/main/lstm_serie_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM com série temporal

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import tensorflow as tf
import sklearn

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
urlTrain = 'https://raw.githubusercontent.com/tmoura/machinelearning/refs/heads/master/datasets/petr4_treinamento.csv'
urlTest =  'https://raw.githubusercontent.com/tmoura/machinelearning/refs/heads/master/datasets/petr4_teste.csv'

base = pd.read_csv(urlTrain) # CSV de Treinamento
base

In [ ]:
base.isnull().sum()

In [5]:
base = base.dropna()

In [ ]:
base.shape

In [7]:
# Pega todas as linhas, mas apenas a coluna "Open"

base_treinamento = base.iloc[:, 1:2].values

base_treinamento

In [ ]:
normalizador = MinMaxScaler(feature_range=(0, 1))

base_treinamento_normalizada = normalizador.fit_transform(base_treinamento)

base_treinamento_normalizada

In [19]:
X = [] # previsores
y = [] # preço real

# Janela deslizante com 90 colunas
for i in range(90, 1242): # 90 preços anteriores
  X.append(base_treinamento_normalizada[i - 90:i,0])
  y.append(base_treinamento_normalizada[i,0])

X, y = np.array(X), np.array(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[0], y[0]

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 100, return_sequences=True, input_shape = (X.shape[1], 1)))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50, return_sequences=True))
regressor.add(Dropout(0.3))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.3))

regressor.add(Dense(units = 1, activation = 'linear'))

regressor.summary()

In [24]:
regressor.compile(optimizer = 'rmsprop', loss = 'mean_squared_error', metrics = ['mean_absolute_error'])

In [ ]:
regressor.fit(X, y, epochs = 100, batch_size = 32)

In [ ]:
base_teste = pd.read_csv(urlTest) # CSV de Teste

base_teste

In [ ]:
base_teste.shape

In [28]:
y_teste = base_teste.iloc[:, 1:2].values

In [ ]:
base.shape

In [30]:
base_completa = pd.concat((base['Open'], base_teste['Open']), axis = 0)

In [ ]:
base_completa.shape

In [ ]:
base_completa

In [ ]:
len(base_completa), len(base_teste), 1264 - 22 - 90

In [ ]:
112 - 90

In [ ]:
base_completa[1152:]

In [34]:
entradas = base_completa[len(base_completa) - len(base_teste) - 90:].values

In [ ]:
entradas

In [ ]:
entradas.shape

In [ ]:
entradas = entradas.reshape(-1, 1)
entradas.shape

In [ ]:
entradas

In [39]:
entradas = normalizador.transform(entradas)

In [ ]:
entradas

In [40]:
X_teste = []
for i in range(90, 112):
  #print(i)
  X_teste.append(entradas[i - 90:i, 0])

In [ ]:
X_teste

In [43]:
X_teste = np.array(X_teste)
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

In [ ]:
X_teste.shape

In [ ]:
previsoes = regressor.predict(X_teste)

In [ ]:
previsoes

In [49]:
previsoes = normalizador.inverse_transform(previsoes)

In [ ]:
previsoes

In [ ]:
y_teste

In [ ]:
previsoes.mean()

In [ ]:
y_teste.mean()

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_teste, previsoes)

In [ ]:
plt.plot(y_teste, color = "red", label = "Preço real")
plt.plot(previsoes, color = "blue", label = "Previsões")
plt.title("Previsão do preço das ações")
plt.xlabel("Tempo")
plt.ylabel("Valor")
plt.legend();